# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [163]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [164]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [165]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#print(data.status_code)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'G/SECKREST)SABLENTQA A( OLDAML NG', 'calculationPrice': 'close', 'open': 136.19, 'openTime': 1698944582031, 'openSource': 'alfociif', 'close': 138.12, 'closeTime': 1688178532743, 'closeSource': 'lioffaci', 'high': 135.61, 'highTime': 1659261722173, 'highSource': 'ri aepe1 cdeumtei5ny ld', 'low': 134.41, 'lowTime': 1639863651666, 'lowSource': 'diap ee 5ilemnreyc td1u', 'latestPrice': 134.88, 'latestSource': 'Close', 'latestTime': 'April 23, 2021', 'latestUpdate': 1642760750280, 'latestVolume': 80821812, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 139.6, 'delayedPriceTime': 1635976652994, 'oddLotDelayedPrice': 135.46, 'oddLotDelayedPriceTime': 1652539836310, 'extendedPrice': 137.33, 'extendedChange': -0.04, 'extendedChangePercent': -0.0003, 'extendedPriceTime': 1660034802448, 'previousClose': 134.34, 'previousVolume': 87513692, 'change': 2.43, 'changePercent': 0.01873, 'volum

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [166]:
price = data['latestPrice']
#price
pe_ratio = data['peRatio']
pe_ratio

37.1

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [167]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [168]:
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        #print(symbol)
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
               symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,139.21,53.12,N/A
1,AAL,22.07,-1.47,N/A
2,AAP,201.96,28.72,N/A
3,AAPL,135.56,37.2,N/A
4,ABBV,114.21,41.67,N/A
...,...,...,...,...
500,YUM,118.72,41.14,N/A
501,ZBH,185.01,-266.69,N/A
502,ZBRA,515.90,54.1,N/A
503,ZION,56.80,8.75,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [169]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,667.27,5.2,N/A
1,ALL,130.03,7.51,N/A
2,UNM,29.43,7.7,N/A
3,EBAY,63.36,7.92,N/A
4,AFL,55.50,8,N/A
5,ZION,56.80,8.75,N/A
6,COO,418.25,9.2,N/A
7,CE,164.60,9.46,N/A
8,KIM,21.21,9.49,N/A
9,CTL,11.00,9.8,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [170]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [171]:
portfolio_input()

Enter the value of your portfolio:1e6


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [172]:
position_size = float(portfolio_size)/len(final_dataframe.index)
#position_size
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = position_size/final_dataframe.loc[row, 'Price']
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,667.27,5.2,29.9729
1,ALL,130.03,7.51,153.811
2,UNM,29.43,7.7,679.579
3,EBAY,63.36,7.92,315.657
4,AFL,55.50,8,360.36
5,ZION,56.80,8.75,352.113
6,COO,418.25,9.2,47.8183
7,CE,164.60,9.46,121.507
8,KIM,21.21,9.49,942.951
9,CTL,11.00,9.8,1818.18


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [173]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#data.status_code
#print(data['AAPL']['advanced-stats'])
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

20.190017609879032


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [174]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
#rv_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        #print(symbol)
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_proft = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [
             symbol,
             data[symbol]['quote']['latestPrice'],
            'N/A',
             data[symbol]['quote']['peRatio'],
            'N/A',
             data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
             data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
             ev_to_ebitda,
            'N/A',
             ev_to_gross_profit,
            'N/A',
            'N/A'
            
        ],
            index = rv_columns
        ),
            ignore_index = True
        )

In [184]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABC,123.88,N/A,-8.00,10.6931,-36.6100,1.18812,0.1310,0.39604,9.178225,13.2673,20.190018,4.05941,5.92079
18,AIG,47.95,N/A,-7.08,11.4851,0.6471,5.34653,0.9759,8.11881,7.328750,6.53465,20.190018,4.05941,7.10891
1,AAL,21.18,N/A,-1.47,14.8515,-1.7000,4.55446,1.0800,9.20792,-3.510828,3.9604,20.190018,4.05941,7.32673
26,ALL,125.91,N/A,7.50,16.1386,1.3600,11.8812,0.8802,6.73267,2.745029,4.55446,20.190018,4.05941,8.67327
17,AFL,55.04,N/A,8.00,16.6337,1.1300,9.20792,1.7000,22.1782,6.725205,5.94059,20.190018,4.05941,11.604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,IDXX,570.00,N/A,84.47,92.4752,76.4500,97.6238,17.8000,96.6337,63.490742,95.4455,20.190018,54.0594,87.2475
150,DXCM,425.00,N/A,85.82,92.6733,22.6200,93.4653,21.7900,98.2178,107.199967,99.4059,20.190018,54.0594,87.5644
65,BKNG,2458.94,N/A,1723.54,100,20.4000,91.8812,15.1400,94.8515,95.380825,98.6139,20.190018,54.0594,87.8812
358,PAYC,413.52,N/A,165.36,96.6337,36.8100,95.4455,29.2200,99.802,97.678048,98.8119,20.190018,54.0594,88.9505


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [176]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,236.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
41,AOS,71.550,N/A,32.73,N/A,6.25,N/A,3.84,N/A,19.770678,N/A,NaN,N/A,N/A
42,APA,18.790,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,APD,300.200,N/A,34.58,N/A,5.26,N/A,7.46,N/A,18.309447,N/A,NaN,N/A,N/A
44,APH,70.800,N/A,17.7,N/A,8.01,N/A,4.9,N/A,21.996723,N/A,NaN,N/A,N/A
45,APTV,146.700,N/A,22.28,N/A,5.08,N/A,3,N/A,47.482835,N/A,NaN,N/A,N/A
46,ARE,185.510,N/A,30.02,N/A,2.32,N/A,14.29,N/A,26.304818,N/A,NaN,N/A,N/A
47,ATO,103.660,N/A,20.98,N/A,1.89,N/A,4.81,N/A,13.689486,N/A,NaN,N/A,N/A
48,ATVI,94.730,N/A,33.9,N/A,4.85,N/A,9.12,N/A,20.421402,N/A,NaN,N/A,N/A
49,AVB,195.850,N/A,33.79,N/A,2.57,N/A,12.08,N/A,24.749362,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [177]:
for column in ['Price', 'Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
    

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [178]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [221]:
from scipy.stats import percentileofscore as score
metrics= {
        'Price-to-Earnings Ratio': 'PE Percentile',
        'Price-to-Book Ratio': 'PB Percentile',
        'Price-to-Sales Ratio': 'PS Percentile',
        'EV/EBITDA': 'EV/EBITDA Percentile',
        'EV/GP': 'EV/GP Percentile',
        }

'''
for metric in metrics.keys():
    for row in rv_dataframe.index:
        if rv_dataframe.loc[row, metric] == None:
           print(rv_dataframe.loc[row, metric])# = 0.0
'''

for metric in metrics.keys():
    #print(metric)
    for row in rv_dataframe.index:
        #print(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])
#rv_dataframe

0     0.18
1     0.38
2      0.2
3     0.24
4     0.42
5     0.45
6     0.45
7     0.06
8     0.34
9      0.3
10    0.16
11    0.32
12     0.6
13     0.1
14    0.48
15    0.14
16    0.12
17    0.04
18    0.26
19       1
20    0.08
21    0.02
22    0.22
23     0.8
24    0.64
25    0.86
26     0.5
27    0.82
28    0.84
29     0.7
30    0.92
31    0.58
32    0.28
33    0.36
34     0.4
35    0.62
36    0.56
37    0.72
38    0.66
39    0.68
40     0.9
41    0.52
42    0.88
43    0.74
44    0.98
45    0.78
46    0.96
47    0.54
48    0.76
49    0.94
Name: PE Percentile, dtype: object
0     0.04
1     0.12
2     0.02
3     0.16
4      0.2
5     0.14
6     0.46
7     0.62
8     0.28
9     0.65
10     0.1
11    0.57
12    0.18
13    0.42
14    0.36
15    0.26
16       1
17    0.53
18    0.94
19     0.6
20    0.32
21    0.44
22    0.82
23    0.57
24    0.96
25    0.08
26     0.4
27    0.48
28     0.5
29    0.34
30     0.7
31    0.98
32    0.22
33     0.9
34    0.65
35    0.74
36    0.77
37     0

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [222]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles =[]
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    #print(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABC,123.88,403.616,-8.00,0.18,-36.6100,0.04,0.1310,0.02,9.178225,0.74,20.190018,0.11,0.218
1,AAL,21.18,2360.72,-1.47,0.38,-1.7000,0.12,1.0800,0.55,-3.510828,0.14,20.190018,0.11,0.26
2,MCK,200.00,250,-7.89,0.2,-68.1500,0.02,0.1315,0.04,7.746186,0.46,20.190018,0.61,0.266
3,AIG,47.95,1042.75,-7.08,0.24,0.6471,0.16,0.9759,0.48,7.328750,0.4,20.190018,0.11,0.278
4,FTI,7.18,6963.79,-0.97,0.42,0.7850,0.2,0.2500,0.08,2.044658,0.16,20.190018,0.61,0.294
5,UNM,29.36,1703,7.50,0.45,0.5412,0.14,0.4515,0.14,3.724939,0.22,20.190018,0.61,0.312
6,ALL,125.91,397.109,7.50,0.45,1.3600,0.46,0.8802,0.42,2.745029,0.2,20.190018,0.11,0.328
7,F,12.42,4025.76,-38.71,0.06,1.6000,0.62,0.3794,0.1,6.176044,0.32,20.190018,0.61,0.342
8,NOV,13.57,3684.6,-2.03,0.34,1.0226,0.28,0.8625,0.4,-12.869044,0.1,20.190018,0.61,0.346
9,MPC,54.00,925.926,-3.70,0.3,1.6200,0.65,0.4999,0.18,-446.631600,0.02,20.190018,0.61,0.352


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [223]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABC,123.88,403.616,-8.00,0.18,-36.6100,0.04,0.1310,0.02,9.178225,0.74,20.190018,0.11,0.218
1,AAL,21.18,2360.72,-1.47,0.38,-1.7000,0.12,1.0800,0.55,-3.510828,0.14,20.190018,0.11,0.26
2,MCK,200.00,250,-7.89,0.2,-68.1500,0.02,0.1315,0.04,7.746186,0.46,20.190018,0.61,0.266
3,AIG,47.95,1042.75,-7.08,0.24,0.6471,0.16,0.9759,0.48,7.328750,0.4,20.190018,0.11,0.278
4,FTI,7.18,6963.79,-0.97,0.42,0.7850,0.2,0.2500,0.08,2.044658,0.16,20.190018,0.61,0.294
5,UNM,29.36,1703,7.50,0.45,0.5412,0.14,0.4515,0.14,3.724939,0.22,20.190018,0.61,0.312
6,ALL,125.91,397.109,7.50,0.45,1.3600,0.46,0.8802,0.42,2.745029,0.2,20.190018,0.11,0.328
7,F,12.42,4025.76,-38.71,0.06,1.6000,0.62,0.3794,0.1,6.176044,0.32,20.190018,0.61,0.342
8,NOV,13.57,3684.6,-2.03,0.34,1.0226,0.28,0.8625,0.4,-12.869044,0.1,20.190018,0.61,0.346
9,MPC,54.00,925.926,-3.70,0.3,1.6200,0.65,0.4999,0.18,-446.631600,0.02,20.190018,0.61,0.352


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [224]:
portfolio_input()

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:2.5e6


In [225]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = position_size/rv_dataframe.loc[row, 'Price']
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABC,123.88,403.616,-8.00,0.18,-36.6100,0.04,0.1310,0.02,9.178225,0.74,20.190018,0.11,0.218
1,AAL,21.18,2360.72,-1.47,0.38,-1.7000,0.12,1.0800,0.55,-3.510828,0.14,20.190018,0.11,0.26
2,MCK,200.00,250,-7.89,0.2,-68.1500,0.02,0.1315,0.04,7.746186,0.46,20.190018,0.61,0.266
3,AIG,47.95,1042.75,-7.08,0.24,0.6471,0.16,0.9759,0.48,7.328750,0.4,20.190018,0.11,0.278
4,FTI,7.18,6963.79,-0.97,0.42,0.7850,0.2,0.2500,0.08,2.044658,0.16,20.190018,0.61,0.294
5,UNM,29.36,1703,7.50,0.45,0.5412,0.14,0.4515,0.14,3.724939,0.22,20.190018,0.61,0.312
6,ALL,125.91,397.109,7.50,0.45,1.3600,0.46,0.8802,0.42,2.745029,0.2,20.190018,0.11,0.328
7,F,12.42,4025.76,-38.71,0.06,1.6000,0.62,0.3794,0.1,6.176044,0.32,20.190018,0.61,0.342
8,NOV,13.57,3684.6,-2.03,0.34,1.0226,0.28,0.8625,0.4,-12.869044,0.1,20.190018,0.61,0.346
9,MPC,54.00,925.926,-3.70,0.3,1.6200,0.65,0.4999,0.18,-446.631600,0.02,20.190018,0.61,0.352


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [226]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [227]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [228]:
column_formats = {
                        'A': ['Ticker', string_template],
                        'B': ['Price', dollar_template],
                        'C': ['Number of Shares to Buy', integer_template],
                        'D': ['Price-to-Earnings Ratio', float_template],
                        'E': ['PE Percentile', percent_template],
                        'F': ['Price-to-Book Ratio', float_template],
                        'G': ['PB Percentile', percent_template],
                        'H': ['Price-to-Sales Ratio', float_template],
                        'I': ['PS Percentile', percent_template],
                        'J': ['EV/EBITDA', float_template],
                        'K': ['EV/EBITDA Percentile', percent_template],
                        'L': ['EV/GP', float_template],
                        'M': ['EV/GP Percentile', percent_template],
                        'N': ['RV Score', float_template]
                  }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [229]:
writer.save()